# Simulate travel time between two points for different transport modes

In [1]:
import haversine as hs
import folium
import math
import sys
from pathlib import Path
from station_utilities import *
from simulations import *

In [2]:
def init_stationfinder() -> StationFinder:
    """
    Initializes the StationFinder utility by loading Citibike and subway graph data.

    Returns:
        StationFinder: An initialized StationFinder object.
    
    Raises:
        SystemExit: If required graph files (.gml) are not found.
    """
    citibike_weekday = '../citibike_weekday_network.gml'
    citibike_weekend = '../citibike_weekend_network.gml'
    subway_graph = '../subway_graph_weekday_weekend.gml'

    citibike_path = citibike_weekday if Path(citibike_weekday).exists() else citibike_weekend

    if not Path(citibike_path).exists():
            print("Error: Citibike graph not found. Please run citibike_processor.py first.")
            sys.exit(1)
        
    if not Path(subway_graph).exists():
        print(f"Error: Subway graph not found at {subway_graph}")
        sys.exit(1)

    print("Initializing StationFinder...")
    try:
        finder = StationFinder(
            citibike_graph_path=citibike_path,
            subway_graph_path=subway_graph
        )
    except Exception as e:
        print(f"Error initializing StationFinder: {e}")
        print("\nIf subway coordinates are not available, you can:")
        print("1. Provide a CSV file with subway coordinates")
        print("2. Download MTA GTFS data manually from: http://web.mta.info/developers/data/nyct/subway/google_transit.zip")
        sys.exit(1)

    return finder

In [3]:
station_finder = init_stationfinder()

Initializing StationFinder...
Successfully loaded 381 subway stations
Successfully loaded 381 subway stations


In [4]:
def simulate(src_hub_center, dst_hub_center, station_finder, radius_km=1.0, num_points=10):
    m = None

    simulator = Simulator(finder=station_finder)

    print('Random starting points:')
    src_hub_points = simulator.generate_random_points(src_hub_center, radius_km=radius_km, num_points=num_points)
    print('Random destination points:')
    dst_hub_points = simulator.generate_random_points(dst_hub_center, radius_km=radius_km, num_points=num_points)

    print("Inserting virtual station at source hub center...")
    simulator.insert_station(('virtual station', src_hub_center))

    # --- Calculate for Bike Transit ---
    bike_travel_time_matrix, bike_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, mode='citibike')

    # --- Calculate for Subway Transit ---
    subway_travel_time_matrix, subway_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, mode='subway')

    results = {
        'bike_travel_time_matrix': bike_travel_time_matrix,
        'bike_route_matrix': bike_route_matrix,
        'subway_travel_time_matrix': subway_travel_time_matrix,
        'subway_route_matrix': subway_route_matrix
    }

    # create a map visualization of the results
    # m = simulator.map_simulation_results(bike_route_matrix, subway_route_matrix, src_hub_points, dst_hub_points)

    return results, m

In [8]:
def calculate_average_travel_times(results):
    bike_tt_matrix, subway_tt_matrix = results['bike_travel_time_matrix'], results['subway_travel_time_matrix']

    # Calculate the mean of all values in the bike travel time matrix
    avg_bike_time_seconds = np.mean(bike_tt_matrix)

    # Calculate the mean of all values in the subway travel time matrix
    avg_subway_time_seconds = np.mean(subway_tt_matrix)

    return avg_bike_time_seconds, avg_subway_time_seconds

In [ ]:
def run_simulation(hubs):
    dst_hub_center = (40.73596286967174, -73.99050483291762)

    bike_results = dict()
    subway_results = dict()

    for hub in hubs:
        results, m = simulate(hub, dst_hub_center, station_finder, radius_km=1.0, num_points=10)
        avg_bike_time, avg_subway_time = calculate_average_travel_times(results)
        bike_results[hub] = f"{math.floor(avg_bike_time / 60)}:{avg_bike_time % 60:.0f}"
        subway_results[hub] = f"{math.floor(avg_subway_time / 60)}:{avg_subway_time % 60:.0f}"

    bike_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Bike': bike_results})
    subway_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Metro': subway_results})

    return bike_avg_times, subway_avg_times

In [20]:
hubs = [(40.6535720712609, -73.931131331664)] # utica ave brooklyn

bikes_results_df, subway_results_df = run_simulation(hubs)

# print("--- Average Travel Times for All Pairs ---")
# print(f"Average Bike Travel Time: {math.floor(avg_bike_time_seconds / 60)} minutes and {avg_bike_time_seconds % 60:.0f} seconds")
# print(f"Average Subway Travel Time: {math.floor(avg_subway_time_seconds / 60)} minutes and {avg_subway_time_seconds % 60:.0f} seconds")

Random starting points:
Generating 10 random points within a 1.0 km radius
Random destination points:
Generating 10 random points within a 1.0 km radius
Inserting virtual station at source hub center...

Inserting new station: virtual station at (40.6535720712609, -73.931131331664)


Calculating subway travel times: 100%|██████████| 10/10 [00:14<00:00,  1.46s/it]



ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
bikes_results_df.head()

,,Average Travel Time By Bike (s)
40.653572,-73.931131,39:21


In [ ]:
subway_results_df.head()

,,Average Travel Time By Metro (s)
40.653572,-73.931131,32:14
